# Representative library virus titers from 042922 virus batch 

This was the very first library rescue that is being used as a representative library titer

In [ ]:
# Imports
import os
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Plotting colors
tol_muted_adjusted = [
    "#000000",
    "#CC6677", 
    "#1f78b4", 
    "#DDCC77", 
    "#117733", 
    "#882255", 
    "#88CCEE",
    "#44AA99", 
    "#999933", 
    "#AA4499", 
    "#EE7733",
    "#CC3311",
    "#DDDDDD",
]

# Seaborn style settings
sns.set(rc={"figure.dpi":300, "savefig.dpi":300})
sns.set_style("ticks")
sns.set_palette(tol_muted_adjusted)

# Suppress warnings
warnings.simplefilter("ignore")

In [ ]:
# Read data
data_frame = pd.read_excel("data/042922_titer_calculations.xlsx")
# Set output directory
out_dir = "figures/"

In [ ]:
# Re-group data
data_frame = data_frame[["Infection_sample", "Titers_(TU/mL)"]]
data_frame["Library"] = data_frame["Infection_sample"].str.contains("_A_")
data_frame["Library"] = data_frame["Library"].map({True: "Library A", False: "Library B"})
data_frame["Infection_sample"] = data_frame["Infection_sample"].str.replace("Library_A_", "")
data_frame["Infection_sample"] = data_frame["Infection_sample"].str.replace("Library_B_", "")
data_frame["Infection_sample"] = data_frame["Infection_sample"].str.replace("_1", "")
data_frame["Infection_sample"] = data_frame["Infection_sample"].str.replace("_2", "")

# Average library titers
data_frame = (
    data_frame
    .groupby(["Infection_sample"])
    .aggregate({"Titers_(TU/mL)" : "mean"})
    .reset_index()
)

# Re-index to have increasing values
data_frame = (
    data_frame
    .reindex([3,2,1,0])
    .reset_index(drop=True)
)
data_frame

In [ ]:
# Plot data
plt.figure(figsize=(2,2))
chart = sns.scatterplot(
    data=data_frame, 
    x="Infection_sample", 
    y="Titers_(TU/mL)", 
    alpha=0.8,
    edgecolor="black",
    linewidth=0.5,
)
chart.set_yscale("log")
chart.set_ylim(0, 50000000) # put at 0 to allow for lowest point to not be cut off
yticks = [10, 1000, 100000, 10000000]
chart.set_yticks(yticks)
chart.set_yticklabels(labels=["$10^1$", "$10^3$", "$10^5$", "$10^7$"], fontsize=8)
xticks = [0, 1, 2, 3]
chart.set_xticks(xticks)
# Re-do x-axis in illustator
x_labels = ["No_Plasmids", "HPs_unconcentrated", "HPs_concentrated",  "HPs+VSVG"]
chart.set_xticklabels(labels=x_labels, rotation=90, horizontalalignment="right", fontsize=8)
chart.set_ylabel("TU/mL", fontsize=8)
chart.set(xlabel=None)

# Change all spines
for axis in ["top", "bottom", "left", "right"]:
    chart.spines[axis].set_linewidth(1)
chart.tick_params(axis="both", length=4, width=1)

sns.despine()

# Make output dir if doesn't exist
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# Save fig
plt.savefig(out_dir + "representative_library_titers.svg")